In [87]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [88]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
%matplotlib inline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold
from sklearn.metrics import roc_curve, auc
from sklearn import metrics
from sklearn.model_selection import train_test_split
import glob
from collections import Counter
from nltk.corpus import stopwords


file_dir = 'C:/Users/danil/Documents/GitHub/Lantern_v2-master/Lantern_v2-master/Data'
file_fol = ['/bully_data','/drug_data','/fight_assault_data','/SH_data','/vandalism_data']

violence_dict = {'Bully_Data':[],'Drug_Data':[],'FA_Data':[],'SH_Data':[],'Vandalism_Data':[]}
violence_keys = ['Bully_Data','Drug_Data','FA_Data','SH_Data','Vandalism_Data']

#Insert text files into violence dictionary if not there already
def insert_to_dict(aDict, name, text_1):
    if not name in aDict:
        aDict[name] = [(" "+text_1)]
    else:
        aDict[name].append(" " + text_1)

#Create violence dictionary
def create_dictionary(aDict, keys, file_directory, file_folder):
    for i in keys:
        place = keys.index(i)
        all_files = glob.glob(file_directory+file_folder[place]+"/*.txt")
        
        for file in (all_files):
            current_file = open(file,'r',encoding = 'latin-1').read()
            insert_to_dict(aDict, i, current_file) 
            
    a_dict = aDict
    
    return a_dict                      

#Preprocessing code
def clean_text(aDict):
    custom_stopwords = ('& et et. al. = % 1 2 3 4 5 6 7 8 9 1â\x81\x844 â« - . ci: 95% â\x80¢ (r').split()

    for type_violence in aDict:
        i = 0
        for paper in aDict[type_violence]:
            words_1 = paper.lower().split()

            #Shorthand notation to longhand
            for word in words_1:
                if 'prv' in word:
                    index = words_1.index(word)
                    words_1[index] = 'physical relationship violence'
                if 'soi' in word:
                    index = words_1.index(word)
                    words_1[index] = 'sociosexual orientation inventory'
                if 'imci' in word:
                    index = words_1.index(word)
                    words_1[index] = 'integrated management of childhood illness'
                if 'mrsdt' in word:
                    index = words_1.index(word)
                    words_1[index] = 'mandatory random student drug testing'
                if 'gsa' in word:
                    index = words_1.index(word)
                    words_1[index] = 'gay-straight alliance'

            #Join words with hyphens
            for word in words_1:
                if '-' in word:
                    index = words_1.index(word)
                    index_1 = index +1 
                    words_1[index] = ''.join(words_1[index:index_1+1])

            #Stopword removal
            filtered_words = [word for word in words_1 if word not in stopwords.words('english')]
            filtered_words = [word for word in filtered_words if word not in custom_stopwords]

            #Remove words with periods in them
            for word in filtered_words:
                if '.' in word:
                    filtered_words.remove(word)

            #Remove words that are less than 2 letters long
            for word in filtered_words:
                if len(word) < 2:
                    filtered_words.remove(word)
                    
            aDict[type_violence][i] = filtered_words
            i += 1
        
    return aDict

#EDA - Create a dictionary/counter of words for each paper in each type of violence
def count_common_words(aDict):
    for type_violence in aDict:
        i = 0
        for words in aDict[type_violence]:
            aDict[type_violence][i] = Counter(words)
            i += 1
    return aDict

            
#EDA - Create a dictionary/counter of words for each type of violence
def count_words_for_each_violence(aDict):     
    corpus_dict = {}

    for type_violence in aDict:
        final_count = Counter()
        for i in range(len(aDict[type_violence])):
            current_count = Counter(aDict[type_violence][i])
            final_count = final_count + current_count

        corpus_dict[type_violence] = final_count
    
    return corpus_dict

#EDA - Break each paper into sentences 
def sentence_split(aDict):
    for type_violence in aDict:
        i = 0
        for words in aDict[type_violence]:
            aDict[type_violence][i] =  words.lower().split('.')
            i += 1
    return aDict

#EDA - count sentences with most words within paper 
def count_top_words_in_sentence(aDict, word_dict, num_most_common):
    
    sentence_dict = sentence_split(aDict)
    
    for type_violence in sentence_dict:
        for paper in sentence_dict[type_violence]:
            index = sentence_dict[type_violence].index(paper)
            words_per_sentence = []
            for i in range(len(paper)):
                
                split_sentence = paper[i].split()
                
                count =  0
                current_most_common = word_dict[type_violence][index].most_common(num_most_common)
                
                for split_word in split_sentence:
                    for j in range(0,num_most_common):
                        if split_word in current_most_common[j][0]:
                            count += 1

                words_per_sentence.append(count) 
                
            index_of_max = words_per_sentence.index(max(words_per_sentence))
            print(paper[index_of_max])
        

#EDA - Create a corpus for all of the words together - maybe have a model dissect through the words and find relationships by
#itself?
def corpus_together(aDict):
    whole_corpus = []
    for type_violence in aDict:
        for array_corpus in aDict[type_violence]:
            whole_corpus += array_corpus
    
    return whole_corpus

In [89]:
violence_dict_fin = create_dictionary(violence_dict, violence_keys, file_dir, file_fol)

In [90]:
violence_dict_fin = clean_text(violence_dict_fin)

In [92]:
x = corpus_together(violence_dict_fin)

232585

In [36]:
word_dict = count_common_words(violence_dict_fin)
word_dict['Bully_Data'][9].most_common(3)

[('school', 90), ('bullying', 84), ('students', 71)]

In [37]:
violence_dict_fin['Bully_Data'][1]

Counter({'motivated': 2,
         'permission': 1,
         'phdâ\x80¡;': 1,
         'studentsâ\x80\x99': 2,
         'victimization': 8,
         'dressing': 1,
         'fell': 3,
         'worries,': 1,
         '3)': 2,
         'â\x80\x932100': 1,
         'incidents)': 1,
         'assessed': 1,
         'feel': 2,
         'essential': 1,
         'perspective-taking': 1,
         'self-reports,or': 1,
         'twenty-twopercent': 1,
         'change': 2,
         'â§rand,': 1,
         'work': 3,
         'internalizing': 1,
         'maintain': 3,
         'cnn.': 1,
         'significant': 3,
         'together': 1,
         'vol.': 1,
         'play': 2,
         'trouble;': 1,
         'adolescence,': 1,
         'services,': 3,
         'consent': 1,
         'wellborn': 1,
         'lot;': 1,
         'distress': 5,
         'shirley': 1,
         'analyses,': 1,
         'behav-ior': 1,
         'larity': 1,
         'report': 7,
         'perry': 2,
         'royal': 

In [38]:
corpus_dict_fin = count_words_for_each_violence(violence_dict_fin)
corpus_dict_fin['Bully_Data'].most_common(3)

[('bullying', 902), ('school', 440), ('students', 336)]

In [80]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [82]:
violence_dict_1 = {'Bully_Data':[],'Drug_Data':[],'FA_Data':[],'SH_Data':[],'Vandalism_Data':[]}
violence_keys_1 = ['Bully_Data','Drug_Data','FA_Data','SH_Data','Vandalism_Data']

violence_dict_1 = create_dictionary(violence_dict_1, violence_keys_1, file_dir, file_fol)
# dict_2 = sentence_split(violence_dict_1)
# dict_2['Bully_Data'][0]

In [83]:
violence_dict_fin = create_dictionary(violence_dict, violence_keys, file_dir, file_fol)
violence_dict_fin = clean_text(violence_dict_fin)
word_dict = count_common_words(violence_dict_fin)

In [84]:
word_dict['Bully_Data'][0].most_common(3)[0][0]

'teachers'

In [85]:
(count_top_words_in_sentence(violence_dict_1, word_dict, 3))


discussion of prior programs
the great teacher program is based on the work of two earlier promising prevention approachesâact early, a program designed to increase positive class- room management strategies,30 and bully busters, a program designed to increase teacher self-efficacy for handling bullies in the classroom

discussion
among youth involved in bullying in a commu- nity sample of ethnically diverse middle school stu- dents, we found that bullies manifest the fewest number of adjustment problems

document type: guides---non-classroom use (055); information analyses---eric information analysis products (iaps) (071); information analyses---eric digests (selected) in full text (073);
descriptors: administrator attitudes, antisocial behavior, bullying, elementary secondary education, intervention, parent attitudes, peer relationship, self esteem, sex differences, student attitudes, student behavior, student problems, teacher attitudes, violence
identifiers: eric digests
###
[re